In [ ]:
import boto3

client = boto3.client('stepfunctions')

In [ ]:
# Note - I set up a role first from the console using the procedure described
# here: http://docs.aws.amazon.com/step-functions/latest/dg/procedure-create-iam-role.html.
# Role ARN is referenced from the envionment
import os
roleARN = os.environ['SFN_ROLE_ARN']
print roleARN

In [ ]:
assemble_response = client.create_activity(
    name='AssemblePizza'
)

assemble_resource = assemble_response['activityArn']


cook_response = client.create_activity(
    name='CookPizza'
)

cook_resource = cook_response['activityArn']

In [ ]:
print assemble_resource
print cook_resource

In [ ]:
response = client.list_activities()
print response

In [ ]:
stateMachine="""
{
  "Comment": "A Hello World example of the Amazon States Language using a Pass state",
  "StartAt": "HelloPizza",
  "States": {
    "HelloPizza": {
      "Type": "Pass",
      "Result": "Order receieved",
      "Next": "AssemblingPizza"
    },
    "AssemblingPizza": {
      "Type": "Task",
      "Resource" : \"""" + assemble_resource + """\",
      "Next": "Cooking"    
    },
    "Cooking": {
      "Type": "Task",
      "Resource" : \"""" + cook_resource + """\",
      "Next": "Ready"    
    },
    "Ready": {
      "Type": "Pass",
      "Result": "Your pizza is ready",
      "End": true    
    }
  }
}
"""

In [ ]:
print stateMachine

In [ ]:
response = client.create_state_machine(
    name='machine1',
    definition=stateMachine,
    roleArn=roleARN
)

print response

In [ ]:
# Can I query for the state machine arn?
def arn_for_machine(name):
    # Are there state machines?
    response = client.list_state_machines()
    if not 'stateMachines' in response:
        return ''
    
    while True:
          
        # Is the machine we are looking for in the list?
        state_machines = response['stateMachines']
        for m in state_machines:
            if m['name'] == name:
                # Found it
                return m['stateMachineArn']
            
        # Are there more to look through?
        if not 'nextToken' in response:
            return ''
        
        next_token = response['nextToken']
        
        response = client.list_state_machines(
            nextToken=next_token
        )

In [ ]:
stateMachineArn = arn_for_machine('machine1')
print stateMachineArn

In [ ]:
import uuid

execution_id = 'sm1' + str(uuid.uuid4())

In [ ]:
start_response = client.start_execution(
    stateMachineArn=stateMachineArn,
    name=execution_id
)

print start_response

In [ ]:
executionArn = start_response['executionArn']
print executionArn

In [ ]:
desc_response = client.describe_execution(
    executionArn=executionArn
)

print desc_response

In [ ]:
get_activity_response = client.get_activity_task(
    activityArn=assemble_resource,
    workerName='worker1'
)

print get_activity_response

In [ ]:
task_token = get_activity_response['taskToken']
print task_token

In [ ]:
ts_response = client.send_task_success(
    taskToken=task_token,
    output='{"status":"pizza assembled, ready to cook"}'
)

print ts_response

In [ ]:
# Now for the cooking
get_activity_response = client.get_activity_task(
    activityArn=cook_resource,
    workerName='worker1'
)

print get_activity_response

In [ ]:
task_token = get_activity_response['taskToken']
print task_token

ts_response = client.send_task_success(
    taskToken=task_token,
    output='{"status":"pizza has been cooked to perfection"}'
)

print ts_response

In [ ]:
desc_response = client.describe_execution(
    executionArn=executionArn
)

print desc_response

In [ ]:
# Clean up
response = client.delete_activity(
    activityArn=assemble_resource
)

print response

In [ ]:
response = client.delete_state_machine(
    stateMachineArn=stateMachineArn
)

print response